In [1]:
import warnings
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import talib
import data
import feature_engineering

warnings.simplefilter(action='ignore', category=FutureWarning)

# Read Prices

In [2]:
df_equity = data.read_excel_sheets()
df_equity['OpenTime'] = pd.to_datetime(df_equity['OpenTime'])
df_equity = df_equity.set_index('OpenTime') 

list_equity = df_equity.Symbol.unique().tolist()
print(f"\nEquity:\n{list_equity}")

crypto = 'data_crypto.xlsx'
df_crypto = (
    pd.read_excel(crypto)
    .drop(columns=['Unnamed: 0'])
)

df_crypto['OpenTime'] = pd.to_datetime(df_crypto['OpenTime'])
df_crypto = df_crypto.set_index('OpenTime') 

list_crypto = df_crypto.Symbol.unique().tolist()
print(f"\nCrypto:\n{list_crypto}")

AAPL
MSFT
NVDA
AMZN
META
GOOGL
BRK.B
GOOG
AVGO
TSLA
SPY_ETF
DAXEX_ETF
CAC40_ETF
NKY225_ETF
DJIA_ETF
EQQQ_ETF
FTSE_ETF

Equity:
['NKY225_ETF', 'AAPL', 'AMZN', 'AVGO', 'BRK.B', 'DJIA_ETF', 'GOOG', 'GOOGL', 'META', 'MSFT', 'NVDA', 'SPY_ETF', 'TSLA', 'DAXEX_ETF', 'CAC40_ETF', 'EQQQ_ETF', 'FTSE_ETF']

Crypto:
['ADAUSDT', 'AVAXUSDT', 'BNBUSDT', 'BTCUSDT', 'DOGEUSDT', 'ETHUSDT', 'SOLUSDT', 'TRXUSDT', 'USDCUSDT', 'XRPUSDT']


**Sample Pairs**

```
Pairs: META & BNBUSDT
Correlation: 0.787

Spread ADF Statistic: -3.3381
p-value: 0.013 (Spread is stationary)

Cointegration Test Statistic: -3.6076
p-value: 0.024 (Both series are cointegrated)

In [3]:
ticker_eqt = 'META'
ticker_cpy = 'BNBUSDT'

close = data.process_pairs_series(ticker_eqt, ticker_cpy, df_equity.reset_index(), df_crypto.reset_index(), 'Close')
close

,META,BNBUSDT
OpenTime,,
2024-10-14 12:30:00,593.1400,590.50
2024-10-14 12:35:00,593.5200,590.10
2024-10-14 12:40:00,594.7200,589.30
2024-10-14 12:45:00,595.3300,588.70
2024-10-14 12:50:00,596.4400,588.30
...,...,...
2025-01-17 18:35:00,614.0775,724.02
2025-01-17 18:40:00,613.7600,724.30
2025-01-17 18:45:00,614.0500,723.72


# Technical Indicators

## Trend-Following Indicators (Momentum-based)

### Exponential Moving Average (EMA)
- Places more weight on recent prices.
- Default Parameters 12, 26

In [4]:
default_values_ema = [12, 26]
list_indicators = []

price_pairs = [ticker_eqt, ticker_cpy]

for ticker in price_pairs:
    for timeperiod in default_values_ema:
        list_indicators.append(
            feature_engineering.exponential_moving_average(close[ticker], ticker, timeperiod)
        )

df_ema = pd.DataFrame(list_indicators).T
df_ema

,META_EMA12,META_EMA26,BNBUSDT_EMA12,BNBUSDT_EMA26
OpenTime,,,,
2024-10-14 12:30:00,NaN,NaN,NaN,NaN
2024-10-14 12:35:00,NaN,NaN,NaN,NaN
2024-10-14 12:40:00,NaN,NaN,NaN,NaN
2024-10-14 12:45:00,NaN,NaN,NaN,NaN
2024-10-14 12:50:00,NaN,NaN,NaN,NaN
...,...,...,...,...
2025-01-17 18:35:00,613.929392,613.986008,723.587291,723.477708
2025-01-17 18:40:00,613.903332,613.969266,723.696939,723.538618
2025-01-17 18:45:00,613.925896,613.975247,723.700486,723.552054


### Moving Average Convergence Divergence (MACD)

- Uses two EMAs to measure trend strength and crossovers.
- Default Parameters:
  - Fast Period: 12
  - Slow Period: 26
  - Signal Period: 9

In [5]:
fast = 12
slow = 26
signal = 9
list_indicators = []

for ticker in price_pairs:
    macd, macdsignal, macdhist = feature_engineering.moving_average_convergence_divergence(
        close[ticker], ticker, fast, slow, signal
    )
    list_indicators.append(macd)

df_macd = pd.DataFrame(list_indicators).T
df_macd

,META_MACD,BNBUSDT_MACD
OpenTime,,
2024-10-14 12:30:00,NaN,NaN
2024-10-14 12:35:00,NaN,NaN
2024-10-14 12:40:00,NaN,NaN
2024-10-14 12:45:00,NaN,NaN
2024-10-14 12:50:00,NaN,NaN
...,...,...
2025-01-17 18:35:00,-0.056616,0.109583
2025-01-17 18:40:00,-0.065935,0.158320
2025-01-17 18:45:00,-0.049351,0.148432


## Momentum Indicators (Oscillators)

### Relative Strength Index (RSI)

- Measures overbought (>70) and oversold (<30) conditions.
- Default Parameters: 14


In [6]:
timeperiod = 14
list_indicators = []

for ticker in price_pairs:
    list_indicators.append(
        feature_engineering.relative_strength_index(close[ticker], ticker, timeperiod)
    )

df_rsi = pd.DataFrame(list_indicators).T
df_rsi

,META_RSI14,BNBUSDT_RSI14
OpenTime,,
2024-10-14 12:30:00,NaN,NaN
2024-10-14 12:35:00,NaN,NaN
2024-10-14 12:40:00,NaN,NaN
2024-10-14 12:45:00,NaN,NaN
2024-10-14 12:50:00,NaN,NaN
...,...,...
2025-01-17 18:35:00,50.800371,53.412849
2025-01-17 18:40:00,48.078678,54.948980
2025-01-17 18:45:00,50.677954,51.184095


In [7]:
ticker_eqt = 'META'
ticker_cpy = 'BNBUSDT'

config = {
    'ema': [12, 26],
    'macd': {
        'fast': 12,
        'slow': 26,
        'signal': 9,
    },
    'rsi': [14, 21],
    'bb': {
        'timeperiod':20, 
        'nbdevup':2, 
        'nbdevdn':2
    },
}

def create_features(ticker_eqt, ticker_cpy, df_equity, df_crypto, config):

    price_pairs = [ticker_eqt, ticker_cpy]
    list_indicators = []

    high = data.process_pairs_series(ticker_eqt, ticker_cpy, df_equity.reset_index(), df_crypto.reset_index(), 'High')
    low = data.process_pairs_series(ticker_eqt, ticker_cpy, df_equity.reset_index(), df_crypto.reset_index(), 'Low')
    close = data.process_pairs_series(ticker_eqt, ticker_cpy, df_equity.reset_index(), df_crypto.reset_index(), 'Close')

    for ticker in price_pairs:

        if 'ema' in config and type(config['ema']) is list:
            for timeperiod in config['ema']:
                list_indicators.append(
                    feature_engineering.exponential_moving_average(close[ticker], ticker, timeperiod)
                )

        if 'macd' in config:
            macd, macdsignal, macdhist = feature_engineering.moving_average_convergence_divergence(
                close[ticker], ticker, config['macd']['fast'], config['macd']['slow'], config['macd']['signal']
            )

            list_indicators.append(macd)

        if 'rsi' in config and type(config['rsi']) is list:
            for timeperiod in config['rsi']:
                list_indicators.append(
                    feature_engineering.relative_strength_index(close[ticker], ticker, timeperiod)
                )

        if 'bb' in config:
            upper, middle, lower = feature_engineering.bollinger_bands(
                close[ticker], ticker, config['bb']['timeperiod'], config['bb']['nbdevup'], config['bb']['nbdevdn']
            )

            list_indicators.append(upper)
            list_indicators.append(middle)
            list_indicators.append(lower)

    return pd.DataFrame(list_indicators).T

In [8]:
create_features(ticker_eqt, ticker_cpy, df_equity.reset_index(), df_crypto.reset_index(), config)

,META_EMA12,META_EMA26,META_MACD,META_RSI14,META_RSI21,META_BBupper,META_BBmiddle,META_BBlower,BNBUSDT_EMA12,BNBUSDT_EMA26,BNBUSDT_MACD,BNBUSDT_RSI14,BNBUSDT_RSI21,BNBUSDT_BBupper,BNBUSDT_BBmiddle,BNBUSDT_BBlower
OpenTime,,,,,,,,,,,,,,,,
2024-10-14 12:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-10-14 12:35:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-10-14 12:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-10-14 12:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-10-14 12:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-17 18:35:00,613.929392,613.986008,-0.056616,50.800371,50.472493,614.888490,613.825295,612.762100,723.587291,723.477708,0.109583,53.412849,53.420232,725.503379,723.7675,722.031621
2025-01-17 18:40:00,613.903332,613.969266,-0.065935,48.078678,48.955457,614.631796,613.756295,612.880794,723.696939,723.538618,0.158320,54.948980,54.344357,725.398232,723.7335,722.068768
2025-01-17 18:45:00,613.925896,613.975247,-0.049351,50.677954,50.385642,614.573134,613.738290,612.903446,723.700486,723.552054,0.148432,51.184095,52.096338,725.302839,723.6885,722.074161


In [9]:
def create_features(ticker_eqt, ticker_cpy, df_equity, df_crypto, config):
    price_pairs = [ticker_eqt, ticker_cpy]
    list_indicators = []

    high = data.process_pairs_series(ticker_eqt, ticker_cpy, df_equity.reset_index(), df_crypto.reset_index(), 'High')
    low = data.process_pairs_series(ticker_eqt, ticker_cpy, df_equity.reset_index(), df_crypto.reset_index(), 'Low')
    close = data.process_pairs_series(ticker_eqt, ticker_cpy, df_equity.reset_index(), df_crypto.reset_index(), 'Close')

    for ticker in price_pairs:
        if 'ema' in config and isinstance(config['ema'], list):
            for timeperiod in config['ema']:
                list_indicators.append(
                    feature_engineering.exponential_moving_average(close[ticker], ticker, timeperiod)
                )

        if 'macd' in config:
            macd, macdsignal, macdhist = feature_engineering.moving_average_convergence_divergence(
                close[ticker], ticker, config['macd']['fast'], config['macd']['slow'], config['macd']['signal']
            )
            list_indicators.append(macd)

        if 'rsi' in config and isinstance(config['rsi'], list):
            for timeperiod in config['rsi']:
                list_indicators.append(
                    feature_engineering.relative_strength_index(close[ticker], ticker, timeperiod)
                )

        if 'bb' in config:
            upper, middle, lower = feature_engineering.bollinger_bands(
                close[ticker], ticker, config['bb']['timeperiod'], config['bb']['nbdevup'], config['bb']['nbdevdn']
            )
            list_indicators.extend([upper, middle, lower])

        if 'atr' in config:
            atr = feature_engineering.average_true_range(high[ticker], low[ticker], close[ticker], ticker, config['atr']['timeperiod'])
            list_indicators.append(atr)

        if 'stoch' in config:
            slowk, slowd = feature_engineering.stochastic_oscillator(
                high[ticker], low[ticker], close[ticker], ticker,
                config['stoch']['fastk_period'], config['stoch']['slowk_period'], config['stoch']['slowd_period']
            )
            list_indicators.extend([slowk, slowd])

        if 'cci' in config:
            cci = feature_engineering.commodity_channel_index(high[ticker], low[ticker], close[ticker], ticker, config['cci']['timeperiod'])
            list_indicators.append(cci)

        if 'willr' in config:
            willr = feature_engineering.williams_percent_r(high[ticker], low[ticker], close[ticker], ticker, config['willr']['timeperiod'])
            list_indicators.append(willr)

    return pd.DataFrame(list_indicators).T

In [10]:
config = {
    'ema': [12, 26],
    'macd': {
        'fast': 12,
        'slow': 26,
        'signal': 9,
    },
    'rsi': [14, 21],
    'bb': {
        'timeperiod': 20,
        'nbdevup': 2,
        'nbdevdn': 2,
    },
    'atr': {
        'timeperiod': 14,
    },
    'stoch': {
        'fastk_period': 14,
        'slowk_period': 3,
        'slowd_period': 3,
    },
    'cci': {
        'timeperiod': 20,
    },
    'willr': {
        'timeperiod': 14,
    },
}

create_features(ticker_eqt, ticker_cpy, df_equity.reset_index(), df_crypto.reset_index(), config)

,META_EMA12,META_EMA26,META_MACD,META_RSI14,META_RSI21,META_BBupper,META_BBmiddle,META_BBlower,META_ATR14,META_StochK14,...,BNBUSDT_RSI14,BNBUSDT_RSI21,BNBUSDT_BBupper,BNBUSDT_BBmiddle,BNBUSDT_BBlower,BNBUSDT_ATR14,BNBUSDT_StochK14,BNBUSDT_StochD3,BNBUSDT_CCI20,BNBUSDT_WILLR14
OpenTime,,,,,,,,,,,,,,,,,,,,,
2024-10-14 12:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-10-14 12:35:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-10-14 12:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-10-14 12:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-10-14 12:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-17 18:35:00,613.929392,613.986008,-0.056616,50.800371,50.472493,614.888490,613.825295,612.762100,0.431433,86.574607,...,53.412849,53.420232,725.503379,723.7675,722.031621,0.631671,48.044976,38.085423,23.742360,-36.466165
2025-01-17 18:40:00,613.903332,613.969266,-0.065935,48.078678,48.955457,614.631796,613.756295,612.880794,0.423295,82.951882,...,54.948980,54.344357,725.398232,723.7335,722.068768,0.606552,70.631497,52.400354,56.520004,-0.000000
2025-01-17 18:45:00,613.925896,613.975247,-0.049351,50.677954,50.385642,614.573134,613.738290,612.903446,0.413774,82.259302,...,51.184095,52.096338,725.302839,723.6885,722.074161,0.604655,78.030737,65.569070,3.419082,-29.441624


In [11]:
config = {
    'ema': [12, 26],
    'macd': {
        'fast': 12,
        'slow': 26,
        'signal': 9,
    },
    'rsi': [14, 21],
    'bb': {
        'timeperiod': 20,
        'nbdevup': 2,
        'nbdevdn': 2,
    },
    'atr': {
        'timeperiod': 14,
    },
    'stoch': {
        'fastk_period': 14,
        'slowk_period': 3,
        'slowd_period': 3,
    },
    'cci': {
        'timeperiod': 20,
    },
    'willr': {
        'timeperiod': 14,
    },
}

feature_engineering.create_features(ticker_eqt, ticker_cpy, df_equity.reset_index(), df_crypto.reset_index(), config)

,META_EMA12,META_EMA26,META_MACD,META_RSI14,META_RSI21,META_BBupper,META_BBmiddle,META_BBlower,META_ATR14,META_StochK14,...,BNBUSDT_RSI14,BNBUSDT_RSI21,BNBUSDT_BBupper,BNBUSDT_BBmiddle,BNBUSDT_BBlower,BNBUSDT_ATR14,BNBUSDT_StochK14,BNBUSDT_StochD3,BNBUSDT_CCI20,BNBUSDT_WILLR14
OpenTime,,,,,,,,,,,,,,,,,,,,,
2024-10-14 12:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-10-14 12:35:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-10-14 12:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-10-14 12:45:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-10-14 12:50:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-17 18:35:00,613.929392,613.986008,-0.056616,50.800371,50.472493,614.888490,613.825295,612.762100,0.431433,86.574607,...,53.412849,53.420232,725.503379,723.7675,722.031621,0.631671,48.044976,38.085423,23.742360,-36.466165
2025-01-17 18:40:00,613.903332,613.969266,-0.065935,48.078678,48.955457,614.631796,613.756295,612.880794,0.423295,82.951882,...,54.948980,54.344357,725.398232,723.7335,722.068768,0.606552,70.631497,52.400354,56.520004,-0.000000
2025-01-17 18:45:00,613.925896,613.975247,-0.049351,50.677954,50.385642,614.573134,613.738290,612.903446,0.413774,82.259302,...,51.184095,52.096338,725.302839,723.6885,722.074161,0.604655,78.030737,65.569070,3.419082,-29.441624
